In [20]:
import pandas as pd
import numpy as np

np.random.seed(1212)

import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers

In [21]:
df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')

In [22]:
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df_features = df_train.iloc[:, 1:785]
df_label = df_train.iloc[:, 0]

X_test = df_test.iloc[:, 0:784]

print(X_test.shape)

(28000, 784)


In [24]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label,
                                                test_size = 0.2,
                                                random_state = 1212)

X_train = X_train.values.reshape(33600, 784) #(33600, 784)
X_cv = X_cv.values.reshape(8400, 784) #(8400, 784)

X_test = X_test.values.reshape(28000, 784)

In [25]:
print((min(X_train[1]), max(X_train[1])))

(np.int64(0), np.int64(255))


In [26]:
X_train = X_train.astype('float32'); X_cv= X_cv.astype('float32'); X_test = X_test.astype('float32')
X_train /= 255; X_cv /= 255; X_test /= 255

num_digits = 10
y_train = keras.utils.to_categorical(y_train, num_digits)
y_cv = keras.utils.to_categorical(y_cv, num_digits)

In [27]:
print(y_train[0]) # 2
print(y_train[3]) # 7

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [28]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [29]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [30]:
model = Model(Inp, output)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_1 (Dense)          │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_2 (Dense)          │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_3 (Dense)          │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_4 (Dense)          │ (None, 200)            │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Layer (Dense)            │ (None, 10)             │         2,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 297,910 (1.14 MB)

 Trainable params: 297,910 (1.14 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
learning_rate = 0.1
training_epochs = 20
batch_size = 100
sgd = optimizers.SGD(learning_rate=learning_rate)

In [32]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [33]:
history1 = model.fit(X_train, y_train,
                     batch_size = batch_size,
                     epochs = training_epochs,
                     verbose = 2,
                     validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 3s - 10ms/step - accuracy: 0.4768 - loss: 1.9335 - val_accuracy: 0.7751 - val_loss: 1.1401
Epoch 2/20
336/336 - 3s - 8ms/step - accuracy: 0.8344 - loss: 0.6785 - val_accuracy: 0.8733 - val_loss: 0.4635
Epoch 3/20
336/336 - 3s - 8ms/step - accuracy: 0.8824 - loss: 0.4113 - val_accuracy: 0.8970 - val_loss: 0.3751
Epoch 4/20
336/336 - 5s - 15ms/step - accuracy: 0.9013 - loss: 0.3409 - val_accuracy: 0.9043 - val_loss: 0.3211
Epoch 5/20
336/336 - 2s - 7ms/step - accuracy: 0.9135 - loss: 0.3013 - val_accuracy: 0.9142 - val_loss: 0.2896
Epoch 6/20
336/336 - 3s - 9ms/step - accuracy: 0.9212 - loss: 0.2725 - val_accuracy: 0.9204 - val_loss: 0.2686
Epoch 7/20
336/336 - 5s - 13ms/step - accuracy: 0.9280 - loss: 0.2498 - val_accuracy: 0.9258 - val_loss: 0.2518
Epoch 8/20
336/336 - 3s - 8ms/step - accuracy: 0.9335 - loss: 0.2304 - val_accuracy: 0.9321 - val_loss: 0.2341
Epoch 9/20
336/336 - 2s - 7ms/step - accuracy: 0.9386 - loss: 0.2131 - val_accuracy: 0.9339 - val_loss: 0.227

In [36]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

# We rely on ADAM as our optimizing methodology
adam = keras.optimizers.Adam(learning_rate=learning_rate)
model2 = Model(Inp, output)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
history2 = model2.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 5s - 15ms/step - accuracy: 0.8949 - loss: 0.3429 - val_accuracy: 0.9539 - val_loss: 0.1521
Epoch 2/20
336/336 - 6s - 19ms/step - accuracy: 0.9613 - loss: 0.1248 - val_accuracy: 0.9546 - val_loss: 0.1472
Epoch 3/20
336/336 - 4s - 12ms/step - accuracy: 0.9738 - loss: 0.0821 - val_accuracy: 0.9683 - val_loss: 0.1049
Epoch 4/20
336/336 - 6s - 17ms/step - accuracy: 0.9807 - loss: 0.0602 - val_accuracy: 0.9707 - val_loss: 0.1026
Epoch 5/20
336/336 - 4s - 13ms/step - accuracy: 0.9852 - loss: 0.0458 - val_accuracy: 0.9738 - val_loss: 0.0892
Epoch 6/20
336/336 - 5s - 16ms/step - accuracy: 0.9893 - loss: 0.0345 - val_accuracy: 0.9731 - val_loss: 0.1085
Epoch 7/20
336/336 - 6s - 17ms/step - accuracy: 0.9896 - loss: 0.0325 - val_accuracy: 0.9676 - val_loss: 0.1167
Epoch 8/20
336/336 - 6s - 18ms/step - accuracy: 0.9912 - loss: 0.0257 - val_accuracy: 0.9744 - val_loss: 0.1029
Epoch 9/20
336/336 - 4s - 13ms/step - accuracy: 0.9923 - loss: 0.0221 - val_accuracy: 0.9712 - val_loss:

In [37]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.01
adam = keras.optimizers.Adam(learning_rate=learning_rate)
model2a = Model(Inp, output)

model2a.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [42]:
history2a = model2a.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                        verbose = 2,
                        validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 - 5s - 15ms/step - accuracy: 0.8988 - loss: 0.3377 - val_accuracy: 0.9511 - val_loss: 0.1656
Epoch 2/20
336/336 - 5s - 16ms/step - accuracy: 0.9610 - loss: 0.1260 - val_accuracy: 0.9637 - val_loss: 0.1305
Epoch 3/20
336/336 - 4s - 10ms/step - accuracy: 0.9738 - loss: 0.0858 - val_accuracy: 0.9701 - val_loss: 0.0976
Epoch 4/20
336/336 - 5s - 14ms/step - accuracy: 0.9814 - loss: 0.0577 - val_accuracy: 0.9723 - val_loss: 0.0898
Epoch 5/20
336/336 - 6s - 18ms/step - accuracy: 0.9847 - loss: 0.0471 - val_accuracy: 0.9699 - val_loss: 0.0989
Epoch 6/20
336/336 - 3s - 10ms/step - accuracy: 0.9892 - loss: 0.0334 - val_accuracy: 0.9719 - val_loss: 0.1108
Epoch 7/20
336/336 - 3s - 10ms/step - accuracy: 0.9879 - loss: 0.0345 - val_accuracy: 0.9706 - val_loss: 0.1102
Epoch 8/20
336/336 - 3s - 9ms/step - accuracy: 0.9933 - loss: 0.0204 - val_accuracy: 0.9726 - val_loss: 0.1166
Epoch 9/20
336/336 - 5s - 15ms/step - accuracy: 0.9933 - loss: 0.0210 - val_accuracy: 0.9752 - val_loss: 

Model 2B

In [43]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

learning_rate = 0.5
adam = keras.optimizers.Adam(learning_rate=learning_rate)
model2b = Model(Inp, output)

model2b.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [44]:
history2b = model2b.fit(X_train, y_train,
                        batch_size = batch_size,
                        epochs = training_epochs,
                            validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8106 - loss: 0.6358 - val_accuracy: 0.9493 - val_loss: 0.1638
Epoch 2/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9588 - loss: 0.1326 - val_accuracy: 0.9649 - val_loss: 0.1128
Epoch 3/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9715 - loss: 0.0848 - val_accuracy: 0.9687 - val_loss: 0.1080
Epoch 4/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9822 - loss: 0.0571 - val_accuracy: 0.9720 - val_loss: 0.0971
Epoch 5/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9863 - loss: 0.0432 - val_accuracy: 0.9724 - val_loss: 0.0965
Epoch 6/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9881 - loss: 0.0350 - val_accuracy: 0.9694 - val_loss: 0.1129
Epoch 7/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9916 - loss: 0.0270 - val_accuracy: 0.9717 - val_loss: 0.1038
Epoch 8/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9930 - loss: 0.0221 - val_accurac

In [45]:
# Input Parameters
n_input = 784 # number of features
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 100
n_hidden_5 = 200
num_digits = 10

In [46]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
x = Dense(n_hidden_5, activation='relu', name = "Hidden_Layer_5")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [48]:
model3 = Model(Inp, output)
model3.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_1 (Dense)          │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_2 (Dense)          │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_3 (Dense)          │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_4 (Dense)          │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_5 (Dense)          │ (None, 200)            │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Layer (Dense)            │ (None, 10)             │         2,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 308,010 (1.17 MB)

 Trainable params: 308,010 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
adam = keras.optimizers.Adam(learning_rate=0.01)

model3.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [50]:
history3 = model3.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.7944 - loss: 0.6698 - val_accuracy: 0.9508 - val_loss: 0.1649
Epoch 2/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9594 - loss: 0.1312 - val_accuracy: 0.9664 - val_loss: 0.1097
Epoch 3/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.9743 - loss: 0.0837 - val_accuracy: 0.9662 - val_loss: 0.1080
Epoch 4/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9793 - loss: 0.0653 - val_accuracy: 0.9714 - val_loss: 0.0979
Epoch 5/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9866 - loss: 0.0458 - val_accuracy: 0.9698 - val_loss: 0.1136
Epoch 6/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9874 - loss: 0.0379 - val_accuracy: 0.9739 - val_loss: 0.0981
Epoch 7/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9897 - loss: 0.0323 - val_accuracy: 0.9705 - val_loss: 0.1184
Epoch 8/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9911 - loss: 0.0247 - val_accura

In [51]:
n_input = 784
n_hidden_1 = 300
n_hidden_2 = 100
n_hidden_3 = 100
n_hidden_4 = 200
num_digits = 10

In [52]:
Inp = Input(shape=(784,))
x = Dense(n_hidden_1, activation='relu', name = "Hidden_Layer_1")(Inp)
x = Dropout(0.3)(x)
x = Dense(n_hidden_2, activation='relu', name = "Hidden_Layer_2")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_3, activation='relu', name = "Hidden_Layer_3")(x)
x = Dropout(0.3)(x)
x = Dense(n_hidden_4, activation='relu', name = "Hidden_Layer_4")(x)
output = Dense(num_digits, activation='softmax', name = "Output_Layer")(x)

In [53]:
model4 = Model(Inp, output)
model4.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_1 (Dense)          │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_2 (Dense)          │ (None, 100)            │        30,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_3 (Dense)          │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Hidden_Layer_4 (Dense)          │ (None, 200)            │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output_Layer (Dense)            │ (None, 10)             │         2,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 297,910 (1.14 MB)

 Trainable params: 297,910 (1.14 MB)

 Non-trainable params: 0 (0.00 B)

In [54]:
model4.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [55]:
history = model4.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = training_epochs,
                    validation_data=(X_cv, y_cv))

Epoch 1/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.6668 - loss: 0.9960 - val_accuracy: 0.9470 - val_loss: 0.1850
Epoch 2/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9258 - loss: 0.2509 - val_accuracy: 0.9580 - val_loss: 0.1450
Epoch 3/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9478 - loss: 0.1808 - val_accuracy: 0.9662 - val_loss: 0.1133
Epoch 4/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9596 - loss: 0.1371 - val_accuracy: 0.9704 - val_loss: 0.1136
Epoch 5/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9637 - loss: 0.1277 - val_accuracy: 0.9702 - val_loss: 0.1030
Epoch 6/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.9680 - loss: 0.1108 - val_accuracy: 0.9711 - val_loss: 0.1026
Epoch 7/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9718 - loss: 0.0962 - val_accuracy: 0.9730 - val_loss: 0.0989
Epoch 8/20
336/336 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9736 - loss: 0.0891 - val_accu

In [56]:
test_pred = pd.DataFrame(model4.predict(X_test, batch_size=200))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'ImageId'
test_pred = test_pred.rename(columns = {0: 'Label'}).reset_index()
test_pred['ImageId'] = test_pred['ImageId'] + 1

test_pred.head()

140/140 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [57]:
test_pred.to_csv('mnist_submission.csv', index = False)

In [63]:
# Save the model using Keras's save function
model.save("my_keras_model.h5")

from google.colab import files
files.download("my_keras_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>